In [2]:
#!pip install aiohttp

In [3]:
import asyncio
import json
import time
from typing import Dict, Any, List, Tuple
import requests
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from itertools import repeat
from aiohttp import ClientSession
import pandas as pd

In [37]:
async def http_get_with_aiohttp(session: ClientSession, url: str, headers: Dict = {}, proxy: str = None, timeout: int = 10) -> (int, Dict[str, Any], bytes):
    response = await session.get(url=url, headers=headers, proxy=proxy, timeout=timeout)

    response_json = None
    try:
        response_json = await response.json(content_type=None)
    except json.decoder.JSONDecodeError as e:
        pass

    return response_json

In [9]:
async def http_get_with_aiohttp_parallel(session: ClientSession, list_of_urls: List[str], headers: Dict = {}, proxy: str = None, timeout: int = 100) -> (List[Tuple[int, Dict[str, Any], bytes]], float):
    t1 = time.time()
    results = await asyncio.gather(*[http_get_with_aiohttp(session, url, headers, proxy, timeout) for url in list_of_urls])
    t2 = time.time()
    t = t2 - t1
    
    return results, t

In [62]:
n = 17
im = pd.read_csv('C:/Users/Cesar/Desktop/Project Work/Data Set/Splited Data/' + str(n) + '_new.csv')
data1 = pd.DataFrame(im[['id']])
data1['id'] = data1['id'].astype(str)
movies = data1['id'].values.tolist()

In [68]:
n = 73
for i in range(1, 72):
    im = pd.read_csv('C:/Users/Cesar/Desktop/Project Work/Data Set/Splited Data/' + str(n) + '_new.csv')
    data1 = pd.DataFrame(im[['id']])
    data1['id'] = data1['id'].astype(str)
    movies = data1['id'].values.tolist()
    API_key = 'a165e3f0026e6a4c02c30bacda76af0b'
    movie_urls = [f'https://api.themoviedb.org/3/movie/'+p+'?api_key='+API_key+'&language=en-US' for p in movies]
    %timeit
    # Benchmark aiohttp
    session = ClientSession()
    results, t = await http_get_with_aiohttp_parallel(session, movie_urls)
    await session.close()
    results
    df1 = pd.json_normalize(results)
    df2 = pd.DataFrame(df1[['id', 'imdb_id', 'budget', 'revenue', 'release_date']])
    df2.to_csv('TMDB ok/' + str(n) + "_newok.csv", index=False)
    n += 1

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E0BFA4CD90>


ClientPayloadError: 400, message='Can not decode content-encoding: gzip'

In [63]:
API_key = 'a165e3f0026e6a4c02c30bacda76af0b'
movie_urls = [f'https://api.themoviedb.org/3/movie/'+i+'?api_key='+API_key+'&language=en-US' for i in movies]

In [64]:
%timeit
# Benchmark aiohttp
session = ClientSession()
results, t = await http_get_with_aiohttp_parallel(session, movie_urls)
await session.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E0B3871490>


ClientPayloadError: 400, message='Can not decode content-encoding: gzip'

In [ ]:
t

In [ ]:
results

In [ ]:
df1 = pd.json_normalize(results)
df1

In [65]:
df2 = pd.DataFrame(df1[['id', 'imdb_id', 'budget', 'revenue', 'release_date']])
df2

,id,imdb_id,budget,revenue,release_date
0,983178.0,tt0008574,0.0,0.0,1917-10-09
1,983179.0,tt0487627,0.0,0.0,1914-07-07
2,983180.0,tt0468201,0.0,0.0,1915-02-20
3,983182.0,None,0.0,0.0,2020-10-10
4,983183.0,None,0.0,0.0,2016-05-10
...,...,...,...,...,...
4736,989412.0,None,0.0,0.0,2015-09-04
4737,989413.0,tt1760933,0.0,0.0,1972-09-07
4738,989414.0,None,0.0,0.0,2008-06-03
4739,989415.0,None,0.0,0.0,2005-06-08


In [66]:
df2.to_csv('TMDB ok/' + str(n) + "_newok.csv", index=False)

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.to_csv('final_data/final_' + str(n) + '.csv')